# Simulation of artery wall dynamics using FEniCS

[FEniCS](https://fenicsproject.org/) is a finite element (FE) solver in Python and is used here to simulate artery wall dynamics under functional hyperaemia. This simulation was used to calculate IPAD under functional hyperaemia [1].

FEniCS is best installed as a standalone Anaconda environment:

Installing ```nb_conda``` in both the ```fenicsproject``` and the base environment adds the option to switch the kernel environment to Jupyter Notebook.

Now we can begin modelling an arteriole using FEniCS. Begin with all your module imports

In [ ]:
from fenics import *
from dolfin import *
from mshr import *

from matplotlib import pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

Next we define useful parameters for units and scaled parameters

In [ ]:
# Units
cm = 1e-2
um = 1e-4 * cm
dyn = 1
pa = 10 * dyn/cm**2
s = 1

# Scaled variables
r0 = 20*um # arteriole radius
R = r0/r0 # dimensionless radius
We = 0.2*R # arteriole wall width
w_ast = 10*um/r0 # width of an astrocyte
gap = 1*um/r0 # gap between astrocytes
Le = 10*R + 5*w_ast + 4*gap # length of the arteriole
tf = 50*s # simulation time

This example simulation will have 5 astrocytes placed along the arteriole wall. They're start and end points along the wall are defined next

In [ ]:
asta1 = Le/2 - 5*w_ast
asta2 = asta1+w_ast
astb1 = asta2+gap
astb2 = astb1+w_ast
astc1 = astb2+gap
astc2 = astc1+w_ast
astd1 = astc2+gap
astd2 = astd1+w_ast
aste1 = astd2+gap
aste2 = aste1+w_ast

Finally, we need the elasticity parameters for the artery wall

In [ ]:
Y = 1.0e6 * pa # Young's modulus
nu = 0.49 # Poisson's ratio
lam = Y*nu/((1+nu)*(1-2*nu)) # First Lame coefficient
mu = Y/(2*(1+nu)) # Second Lame coefficient
Mu = mu/mu # dimensionless Lame coefficient
Lam = lam/mu # dimensionless Lame coefficient

The displacement for the arteriole wall is obtained from the simulations of the neurovascular unit (NVU) following [2].

In [ ]:
disp = np.loadtxt('./nvu/disp.csv', delimiter=',')/r0 # read displacement from data 
nt = disp.shape[0] # number of time steps
dt = tf/nt

To set up FEniCS we need to set up the geometry and mesh using meshr

In [ ]:
N = 512
deg = 2
elem = "Lagrange"
geom = Rectangle(Point(0, 0), Point(We, Le))
mesh = generate_mesh(geom, N)

We will need three function spaces for the simulation

In [ ]:
V = VectorFunctionSpace(mesh, elem, deg)
W = FunctionSpace(mesh, elem, deg)
WW = TensorFunctionSpace(mesh, elem, deg)

Astrocytes on the arteriole wall are defined as Dirichlet boundary conditions on the top boundary of the arteriole wall with prescribed displacement obtained from ```disp```. The bottom boundary of the arteriole wall is allowed to move freely. The side boundaries of the rectangular arteriole geometry are fixed to zero displacement. 

In [ ]:
def astr_a(x, on_boundary):
    return near(x[0], We) and (x[1] < asta2 and x[1] > asta1)

def astr_b(x, on_boundary):
    return near(x[0], We) and (x[1] < astb2 and x[1] > astb1)

def astr_c(x, on_boundary):
    return near(x[0], We) and (x[1] < astc2 and x[1] > astc1)

def astr_d(x, on_boundary):
    return near(x[0], We) and (x[1] < astd2 and x[1] > astd1)

def astr_e(x, on_boundary):
    return near(x[0], We) and (x[1] < aste2 and x[1] > aste1)

def fixed_boundary(x, on_boundary):
    return near(x[1], 0) or near(x[1], Le)

disps = Expression(('d', '0'), d=disp[0], degree=deg)
bc0 = DirichletBC(V, Constant((0, 0)), fixed_boundary)
bc1 = DirichletBC(V, disps, astr_a, method='pointwise')
bc2 = DirichletBC(V, disps, astr_b, method='pointwise')
bc3 = DirichletBC(V, disps, astr_c, method='pointwise')
bc4 = DirichletBC(V, disps, astr_d, method='pointwise')
bc5 = DirichletBC(V, disps, astr_e, method='pointwise')
bcs = [bc0, bc1, bc2, bc3, bc4, bc5]

Next we define functions for stress and strain in the PDE

In [ ]:
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def sigma(u):
    return Lam*nabla_div(u)*Identity(d) + 2*Mu*epsilon(u)

With all parameters and variables in place, we can set up the variational Problem in FEniCS

In [ ]:
u = TrialFunction(V)
d = u.geometric_dimension()
v = TestFunction(V)
f = Constant((0, 0))
a = inner(sigma(u), epsilon(v))*dx
L = dot(f, v)*dx

Solutions should be stored in a VTK stack

In [ ]:
ufile = File('./fenics_artery/u.pvd')
sfile = File('./fenics_artery/s.pvd')

Finally, the simulation itself is run in a loop over time. Each time step the current displacement gets updated on the boundary conditions and the variational problem is solved.

In [ ]:
u = Function(V)
t = 0.0
for i in range(nt):
    disps.d = disp[i] # update displacement in all astrocyte boundary conditions
    solve(a == L, u, bcs)
    u.rename('u (um/s)', 'u (um/s)')
    
    # Obtain principal stress component in radial direction
    sigma_w = project(sigma(u)*Identity(d), WW)
    sigma_r = project(sigma_w[0, 0], W)
    sigma_r.rename('sigma (Pa)', 'sigma (Pa)')
    
    # Save to file and plot solution
    ufile << (u, t)
    sfile << (sigma_r, t)
    
    t += dt

Because FEniCS does not reliably execute in Jupyter Notebooks this file serves more as an explanation of the accompanying code file. The data has been generated using this file.

The following steps to retrieve pressure within the basement membrane were carried out using Paraview. Load the s.pvd file into Paraview and create a line along the length of the artery. Export the stress values along this line. This data is saved in stress.csv.

## References

[1] Diem AK, MacGregor Sharp M, Gatherer M, Bressloff NW, Carare RO and Richardson G (2017) Arterial Pulsations cannot Drive Intramural Periarterial Drainage: Significance for Aβ Drainage. Frontiers in Neuroscience 11:475. doi: https://doi.org/10.3389/fnins.2017.00475

[2] Diem AK (2017) [Re] A bidirectional model for communication in the neurovascular unit. ReScience 3(1): 9. url: https://github.com/ReScience-Archives/Diem-2017/blob/master/article/Diem-2017.pdf